# Examen de cómputo matricial equipo SVD

**Fecha**

19 de Abril de 2020

**Objetivo**

Programar en el lenguaje R el método de eliminación por bloques para la solución de un sistema de ecuaciones $$Ax=b$$ mediante el método de descomposición en valores singulares (DVS o SVD por siglas en inglés), a través del Algoritmo One-Sided Jacobi.

**Especificaciones de ambiente común de trabajo**

Para trabajar y a efecto de tener un entorno común de trabajo para el desarrollo del proyecto, por favor emplear la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 1.1 Generación de índices

In [1]:
indices <- function(n) {
  # Crea una lista de tamaño (n-1)n/2 con pares de índices de la siguiente
  #  manera: (1,2),..,(1,n),(2,3),..,(2,n),...,(n-1,n)
  # Args: 
  #    n: número entero postivo 
  #       se refiere al número de columnas
  #Returns:
  #    lista con pares de índices
    a <- NULL
    b <- NULL
    indices <- NULL
    for (i in 1:(n-1)){
    a <- append(a,rep(i,n-i))
    b <- append(b,seq(i+1,n))    
    }
    for(i in 1:round(n*(n-1)/2))
    indices[[i]] <- list(c(a[i], b[i]))
    indices


}

### 1.2 Verificación de ortogonalidad entre vectores

### 1.3 Función signo

In [1]:
signo<-function(x) {
  # Indica el signo de un número
  # Args: 
  #    X (numeric): número a revisar
  #Returns:
  #    TRUE si el número es positivo o cero
  #    FALSE si el número es negativo
  
  ifelse(x<0,FALSE,TRUE)
  }

### 1.4 Solver dada descomposición SVD

## 2. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 One-sided Jacobi numerical aproximación

In [5]:
#algoritmo sin forma funcional

#parametros de entrada
A<-matrix(c(3,2,2,3,2,-2),nrow=2)
TOL<-10**-8
maxsweep<-20

#dimensiones
n<-dim(A)[2] #numero de columnas
m<-dim(A)[1] #numero de filas
nmax<-n*(n-1)/2

#inicialza valores del ciclo
ak<-A
vk<-diag(n)
num_col_ortogonal<-0
k<-0

while(k<=maxsweep & num_col_ortogonal<nmax){
  num_col_ortogonal<-0
  
  #falta implementar que los ciclos se recorran sobre la salida de la funcion indices
  for(i in seq(1,n-1)){
    for(j in seq(i+1,n)){
      col_j<-ak[,j]
      col_i<-ak[,i]
    
      #comprueba ortogonalidad  
      if(son_ortogonales(col_i,col_j,TOL)){
        num_col_ortogonal<-num_col_ortogonal+1
      }
      else{
        #calcula coeficientes de la matriz
        a<-sum(col_i*col_i)
        b<-sum(col_j*col_j)
        c<-sum(col_i*col_j)
        
        #calcula la rotacion givens que diagonaliza
        epsilon<-(b-a)/(2*c)
        t<-signo(epsilon)/(abs(epsilon)+sqrt(1+epsilon**2))
        cs<-1/sqrt(1+t**2)
        sn<-cs*t
        
        #actualiza las columnas de la matriz ak
        for(l in seq(1,m)){
          temp<-ak[l,i]
          ak[l,i]<-cs*temp-sn*ak[l,j]
          ak[l,j]<-sn*temp+cs*ak[l,j]
        }
        
        #actualiza las columnas de la matriz vk
        for(l in seq(1,n)){
          temp<-vk[l,i]
          vk[l,i]<-cs*temp-sn*vk[l,j]
          vk[l,j]<-sn*temp+cs*vk[l,j]
          
        }
      }
    }
  }
  k<-k+1
}

In [ ]:
La matriz v esta guardada como vk
Los eigens son las normas de las columnas de a
falta sacar la matriz U que tambien sale de A

In [14]:
norma<-function(x){
    norm<-sqrt(sum(x*x))
    norm<-ifelse(norm<10**-8,0,norm)
}

entradas_sigma<-apply(ak,MARGIN=2,norma)
sigma<-diag(entradas_sigma)
sigma

0,0,0
0,5,0
0,0,3


In [2]:
#funciones auxilieres que aun no funcionan bien arriba
signo<-function(x){
  ifelse(x<0,-1,1)
}

son_ortogonales<-function(col_i,col_j,TOL){
  if(sum(col_i*col_j)/(sqrt(sum(col_i*col_i))*sqrt(sum(col_j*col_j)))<TOL){
    ret<-TRUE
  }
  else{
    ret<-FALSE
  }
  ret
}


### 2.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm